In [2]:
# Install access to Ensembl biomart
!pip install pybiomart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.2 MB/s eta 0:00:00


# IMPORT STATEMENTS

In [3]:
import pandas as pd
import pybiomart as mart

# DATA PREPARATION
Attachment of gene annotations to count data and formatting in preparation for DESeq2.


## HUMAN GENE ANNOTATION
Retrieve from Ensembl and remove duplicated IDs to create gene information table to merge with counts table

In [ ]:
# RETRIEVE INFO FROM ENSEMBL AND REMOVE UNWANTED GENE TYPES
dataset = mart.Dataset(name='hsapiens_gene_ensembl',
                  host='http://dec2021.archive.ensembl.org') # ARCHIVAL ENSEMBL TO MATCH ALIGNMENT
# dataset.list_attributes()
filtered = dataset.query(attributes=["ensembl_gene_id","hgnc_symbol",
                                     "gene_biotype","chromosome_name",
                                     "description"],
                         filters={"biotype":["protein_coding","lincRNA"]})
#filtered.head()

# REMOVE DUPLICATED GENE IDS
print("Number of genes in human dataset before duplicate removal: " +
      str(filtered.shape[0]))
print("Duplicated gene IDs to be removed except for the first instance:")
display(filtered[filtered["Gene stable ID"].duplicated()])
filtered.drop_duplicates(subset = "Gene stable ID",
                         inplace = True) # DROP DUPLICATES IN PLACE
print("Number of genes in human dataset after duplicate removal: " +
      str(filtered.shape[0]))

## PLASMODIUM GENE ANNOTATION
Retrieve Plasmodium gene information and remove duplicated IDs. Merge information from alignments with gene symbols from PlasmoDB

In [ ]:
# READ IN PF GENE ANNOTATIONS USED IN ALIGNMENT
pf_genes = pd.read_csv("/content/drive/MyDrive/2023/Pf_ASM276v2_Genes.txt",
                       header = 0,
                       sep = "\t")
print("Number of annotated genes: " + str(pf_genes.shape[0]))

# READ IN PF GENE SYMBOLS FROM PLASMODB
pf_symbols = pd.read_csv("/content/drive/MyDrive/2023/PfGeneNames_plasmodb.csv",
                         header = 0)
print("Number of genes with symbols: " + str(pf_symbols.shape[0]))
#print(pf_genes.columns)
#print(pf_symbols.columns)

Number of genes with info: 5377
Number of genes with symbols: 5342
Index(['Gene stable ID', 'Gene description', 'Chromosome/scaffold name',
       'Gene start (bp)', 'Gene end (bp)', 'Strand', 'Gene name', 'Gene type',
       'UniProtKB-Gene Ontology Annotation ID', 'UniProtKB/Swiss-Prot ID'],
      dtype='object')
Index(['Gene ID', 'source_id', 'Input ID', 'Gene Name or Symbol'], dtype='object')


### REMOVE DUPLICATED GENE IDS

In [ ]:
# REMOVE DUPLICATED GENE IDS FROM ANNOTATIONS USED IN ALIGNMENT
print("Number of genes in pf dataset 1 before duplicate removal: " +
      str(pf_genes.shape[0]))
print("Duplicated gene IDs to be removed except for the first instance:")
display(pf_genes[pf_genes["Gene stable ID"].duplicated()])
pf_genes.drop_duplicates(subset = "Gene stable ID",
                         inplace = True) # DROP DUPLICATES IN PLACE
print("Number of genes in pf dataset 1 after duplicate removal: " +
      str(pf_genes.shape))

# REMOVE DUPLICATED GENE IDS FROM PLASMODB GENE SYMBOLS
print("Number of genes in pf dataset 2 before duplicate removal: " +
      str(pf_symbols.shape[0]))
print("Duplicated gene IDs to be removed except for the first instance:")
display(pf_symbols[pf_symbols["Gene ID"].duplicated()])
pf_symbols.drop_duplicates(subset = "Gene ID",
                         inplace = True) # DROP DUPLICATES IN PLACE
print("Number of genes in pf dataset 2 after duplicate removal: " +
      str(pf_symbols.shape))

Number of genes in pf dataset 2 before duplicate removal: 5342
Duplicated gene IDs to be removed except for the first instance:


,Gene ID,source_id,Input ID,Gene Name or Symbol
45,PF3D7_0105400,PF3D7_0105400.2,"PF3D7_0105400.2, PF3D7_0105400.1",NaN
75,PF3D7_0108400,PF3D7_0108400.2,"PF3D7_0108400.1, PF3D7_0108400.2",NaN
159,PF3D7_0202600,PF3D7_0202600.2,"PF3D7_0202600.2, PF3D7_0202600.1",NaN
190,PF3D7_0205700,PF3D7_0205700.2,"PF3D7_0205700.2, PF3D7_0205700.1",NaN
201,PF3D7_0206900,PF3D7_0206900.2,"PF3D7_0206900.1, PF3D7_0206900.2",MSP5
...,...,...,...,...
4764,PF3D7_1419800,PF3D7_1419800.2,"PF3D7_1419800.1, PF3D7_1419800.2",GR
4828,PF3D7_1427400,PF3D7_1427400.2,"PF3D7_1427400.2, PF3D7_1427400.1",NaN
4843,PF3D7_1428800,PF3D7_1428800.2,"PF3D7_1428800.2, PF3D7_1428800.1",NaN
4913,PF3D7_1435700,PF3D7_1435700.2,"PF3D7_1435700.2, PF3D7_1435700.1",NaN


Number of genes in pf dataset 2 after duplicate removal: (5271, 4)


### MERGE PLASMODIUM ANNOTATIONS AND GENE SYMBOLS
Create final gene information table to be merged with counts table

In [ ]:
# MERGE ANNOTATIONS AND GENE SYMBOLS
pf_merge = pf_genes.merge(right = pf_symbols,
                          left_on = "Gene stable ID",
                          right_on = "Gene ID",
                          how = "left")
print(pf_merge.shape[0])

# DROP EXTRA COLUMNS
pf_merge = pf_merge[["Gene stable ID","Gene description",
                     "Gene Name or Symbol","Chromosome/scaffold name",
                     "Gene type"]]

5358


# PREPARE FINAL TABLES TO USE WITH DESEQ2
Filter the counts table to include only protein-coding, lncRNA and filter the gene info tables to include only genes in the final counts table.

In [ ]:
# READ COUNTS TABLE
my_counts = pd.read_csv("/content/drive/MyDrive/2023/220103_HsPf_2020.txt",
                        header = 0,
                        sep = "\t")
my_genes = my_counts["Gene.ID"]

# SEPARATE HUMAN AND PF GENE COUNT TABLES
human = my_genes[my_genes.str.startswith("ENSG")] # ALL HUMAN
pf = my_genes[my_genes.str.startswith("PF3D7_")] #ALL PF
my_human = my_counts[my_counts["Gene.ID"].isin(human)] # FILTER BY HUMAN
my_pf = my_counts[my_counts["Gene.ID"].isin(pf)] # FILTER BY PF
print(my_human.shape[0])
print(my_pf.shape[0])

# FILTER COUNT TABLES BY GENES LISTED IN THE INFO TABLES
# TO REMOVE GENES THAT ARE NOT PROTEIN-CODING OR LNCRNA
final_human = my_human[
    my_human["Gene.ID"].isin(filtered["Gene stable ID"])
    ]
final_pf = my_pf[
    my_pf["Gene.ID"].isin(pf_merge["Gene stable ID"])
    ]
print(final_human.shape[0])
print(final_pf.shape[0])

# WRITE INTERMEDIATE GENE COUNT TABLES TO FILE
#final_human.to_csv("/content/drive/MyDrive/2023/HumanGeneCountTable_v1.csv",
#                   index = False,
#                   header = True)
#final_pf.to_csv("/content/drive/MyDrive/2023/PfGeneCountTable_v1.csv",
#                index = False,
#                header = True)

### CREATE FINAL GENE INFO TABLES
These tables only contain genes that are included in the final counts tables (ie protein-coding, lncRNA)

In [ ]:
# FILTER GENE INFO TO MATCH GENE COUNT TABLES
pf_edit = pf_merge[pf_merge["Gene stable ID"].isin(final_pf["Gene.ID"])]
human_edit = filtered[filtered["Gene stable ID"].isin(final_human["Gene.ID"])]

# WRITE EDITED GENE INFO TO FILE
human_edit.to_csv("/content/drive/MyDrive/2023/HumanGeneInfoTable_v1.csv",
                  index = False,
                  header = True)
pf_edit.to_csv("/content/drive/MyDrive/2023/PfGeneInfoTable_v1.csv",
               index = False,
               header = True)

## CREATE FINAL COUNTS TABLES

In [ ]:
# REMOVE REPLICATES THAT FAILED QC
bad_reps = ["ProE_Infected_12",
            "BasE_Uninfected_12",
            "BasE_Infected_21",
            "PolyE_Infected_32",
            "OrthoE_Media_31",
            "OrthoE_Infected_31",
            "OrthoE_Infected_32",
            "PolyE_Media_32",
            "RSC",
            "LTC"]
final_human.drop(bad_reps,axis=1,inplace = True) # REMOVING IN PLACE
final_pf.drop(bad_reps,axis = 1, inplace = True) # REMOVING IN PLACE

# REORDER COLUMNS BY CELL TYPE AND CONDITION
my_cols = final_human.columns.tolist()
new_order = ["Gene.ID"]
for cell in ["ProE","BasE","PolyE","OrthoE"]:
  for con in ["Media","Mock","Uninfected","Infected"]:
    for rep in ["11","12","21","22","31","32"]:
      my_name = cell + "_" + con + "_" + rep
      if my_name in my_cols:
        new_order.append(my_name)
print(my_cols)
print(new_order)
print(len(my_cols))
print(len(new_order))

# REORDER COUNTS TABLES
final_human_reo = final_human.loc[:,new_order]
final_pf_reo = final_pf.loc[:,new_order]

# RENAME COLUMNS TO FIX CONDITION NAMES AND CELL TYPE NAMES TO BE CONSISTENT
new_names = {}
for col in new_order:
  col_new = col.replace("_",".")
  if "BasE" or "Mock" in col:
    col_new = col_new.replace("BasE","BasoE")
    col_new = col_new.replace("Mock", "SRS")
  new_names[col] = col_new
print(new_names)
renamed_human = final_human_reo.rename(new_names, axis=1)
renamed_pf = final_pf_reo.rename(new_names, axis = 1)

# WRITE TO FILE THE FINAL COUNTS TABLES FOR ANALYSIS
renamed_human.to_csv("/content/drive/MyDrive/2023/HumanGeneCountsTable_v2.csv",
               index = False,
               header = True)
renamed_pf.to_csv("/content/drive/MyDrive/2023/PfGeneCountsTable_v2.csv",
               index = False,
               header = True)

<ipython-input-17-34ba5ddf26cb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_human.drop(bad_reps,axis=1,inplace = True)
<ipython-input-17-34ba5ddf26cb>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_pf.drop(bad_reps,axis = 1, inplace = True)


## FIND THE ALIGNED READ TOTALS AND PERCENTAGES FOR HU AND PF
This requires going back to the original counts table and applying the same removal of failed replicates and renaming as above so that the totals can be calculated from the unfiltered table.

## UPDATE THE ORIGINAL COUNTS TABLE
Remove failed replicates, reorder, and rename as above

In [ ]:
## REMOVE UNWANTED COLUMNS
# Changes are made in place
bad_reps = ["ProE_Infected_12",
            "BasE_Uninfected_12",
            "BasE_Infected_21",
            "PolyE_Infected_32",
            "OrthoE_Media_31",
            "OrthoE_Infected_31",
            "OrthoE_Infected_32",
            "PolyE_Media_32",
            "RSC",
            "LTC"]
my_counts.drop(bad_reps,axis=1,inplace = True)

## REORDER COLUMNS BY CELL TYPE AND CONDITION
my_cols = my_counts.columns.tolist()
new_order = ["Gene.ID"]
for cell in ["ProE","BasE","PolyE","OrthoE"]:
  for con in ["Media","Mock","Uninfected","Infected"]:
    for rep in ["11","12","21","22","31","32"]:
      my_name = cell + "_" + con + "_" + rep
      if my_name in my_cols:
        new_order.append(my_name)

my_counts = my_counts.loc[:,new_order]

## RENAME COLUMNS TO BE CONSISTENT WITH PAPER
new_names = {}
for col in new_order:
  col_new = col.replace("_",".")
  if "BasE" or "Mock" in col:
    col_new = col_new.replace("BasE","BasoE")
    col_new = col_new.replace("Mock", "SRS")
  new_names[col] = col_new
print(new_names)
my_counts = my_counts.rename(new_names, axis=1)

## SEPARATE INTO TABLES FOR HUMAN AND PF
my_human = my_counts[my_counts["Gene.ID"].isin(human)] # FILTER BY HUMAN
my_pf = my_counts[my_counts["Gene.ID"].isin(pf)] # FILTER BY PF

## FIND ALIGNED READ TOTALS
Sum the total aligned reads from human and Pf and find the percentage of total aligned reads for each species.

In [ ]:
my_total = my_counts.sum(numeric_only=True)
my_human_total = my_human.sum(numeric_only=True)
my_pf_total = my_pf.sum(numeric_only=True)
#print(my_total.shape)
#print(my_human_total.shape)
#print(my_pf_total.shape)

## MAKE FINAL TABLE WITH TOTALS AND PERCENTAGES
my_totals = pd.concat([my_total,my_human_total,my_pf_total], axis = 1)
my_totals.columns = ["Total gene counts","H. sapiens gene counts", "P. falciparum gene counts"]
my_totals["H. sapiens gene counts (% of total)"] = my_totals["H. sapiens gene counts"]/my_totals["Total gene counts"] * 100
my_totals["P. falciparum gene counts (% of total)"] = my_totals["P. falciparum gene counts"]/my_totals["Total gene counts"] * 100

## SAVE TO FILE
my_totals.to_csv("/content/drive/MyDrive/2023/GeneCountsTotals.csv",
               index = True,
               header = True)